# Using `prompto` with Gemini

In [1]:
from prompto.settings import Settings
from prompto.experiment import Experiment
from dotenv import load_dotenv
import os

When using `prompto` to query models from the Gemini API, lines in our experiment `.jsonl` files must have `"api": "gemini"` in the prompt dict. 

## Environment variables

For the [Gemini API](../../docs/models.md#gemini), there are two environment variables that could be set:
- `GEMINI_API_KEY`: the API key for the Gemini API

As mentioned in the [model docs](../../docs/models.md#model-specific-environment-variables), there are also model-specific environment variables too which can be utilised. In particular, when you specify a `model_name` key in a prompt dict, one could also specify a `GEMINI_API_KEY_model_name` environment variable to indicate the API key used for that particular model (where "model_name" is replaced to whatever the corresponding value of the `model_name` key is). We will see a concrete example of this later.

To set environment variables, one can simply have these in a `.env` file which specifies these environment variables as key-value pairs:
```
GEMINI_API_KEY=<YOUR-GEMINI-KEY>
```

If you make this file, you can run the following which should return `True` if it's found one, or `False` otherwise:

In [2]:
load_dotenv(dotenv_path=".env")

True

Now, we obtain those values. We raise an error if the `GEMINI_API_KEY` environment variable hasn't been set:

In [3]:
GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")
if GEMINI_API_KEY is None:
    raise ValueError("GEMINI_API_KEY is not set")

If you get any errors or warnings in the above two cells, try to fix your `.env` file like the example we have above to get these variables set.

## Types of prompts

With the Gemini API, the prompt (given via the `"prompt"` key in the prompt dict) can take several forms:
- a string: a single prompt to obtain a response for
- a list of strings: a sequence of prompts to send to the model
    - this is useful in the use case of simulating a conversation with the model by defining the user prompts sequentially
- a list of dictionaries with keys "role" and "parts", where "role" is one of "user", "model", or "system" and "parts" is the message
    - this is useful in the case of passing in some conversation history or to pass in a system prompt to the model
    - note that only the prompt in the list can be a system prompt, and the rest must be user or model prompts

We have created an input file in [data/input/gemini-example.jsonl](./data/input/gemini-example.jsonl) with an example of each of these cases as an illustration.

In [4]:
settings = Settings(data_folder="./data", max_queries=30)
experiment = Experiment(file_name="gemini-example.jsonl", settings=settings)

We set `max_queries` to 30 so we send 30 queries a minute (every 2 seconds).

In [5]:
print(settings)

Settings: data_folder=./data, max_queries=30, max_attempts=3, parallel=False
Subfolders: input_folder=./data/input, output_folder=./data/output, media_folder=./data/media


In [6]:
len(experiment.experiment_prompts)

6

We can see the prompts that we have in the `experiment_prompts` attribute:

In [7]:
experiment.experiment_prompts

[{'id': 0,
  'api': 'gemini',
  'model_name': 'gemini-1.5-flash',
  'prompt': 'How does technology impact us?',
  'safety_filter': 'none',
  'parameters': {'candidate_count': 1,
   'temperature': 1,
   'max_output_tokens': 100}},
 {'id': 1,
  'api': 'gemini',
  'model_name': 'gemini-1.0-pro',
  'prompt': 'How does technology impact us?',
  'safety_filter': 'few',
  'parameters': {'candidate_count': 1,
   'temperature': 1,
   'max_output_tokens': 100}},
 {'id': 2,
  'api': 'gemini',
  'model_name': 'gemini-1.5-flash',
  'prompt': ['How does international trade create jobs?',
   'I want a joke about that'],
  'safety_filter': 'some',
  'parameters': {'candidate_count': 1,
   'temperature': 1,
   'max_output_tokens': 100}},
 {'id': 3,
  'api': 'gemini',
  'model_name': 'gemini-1.5-flash',
  'prompt': [{'role': 'system',
    'parts': 'You are a helpful assistant designed to answer questions briefly.'},
   {'role': 'user',
    'parts': 'What efforts are being made to keep the hakka language

- In the first prompt (`"id": 0`), we have a `"prompt"` key which is a string and specify a `"model_name"` key to be "gemini-1.5-flash"
- In the second prompt (`"id": 1`), we have a `"prompt"` key is also a string but we specify a `"model_name"` key to be "gemini-1.0-pro".
- In the third prompt (`"id": 2`), we have a `"prompt"` key which is a list of strings.
- In the fourth prompt (`"id": 3`), we have a `"prompt"` key which is a list of dictionaries. These dictionaries have a "role" and "parts" key. This acts as passing in a system prompt. Here, we just have a system prompt before a user prompt.
- In the fifth prompt (`"id": 4`), we have a `"prompt"` key which is a list of dictionaries. These dictionaries have a "role" and "parts" key but here, we have a user prompt and then a system prompt. As mentioned above, only the first prompt in the list can be a system prompt. We should get an error for this particular prompt.
- In the sixth prompt (`"id": 5`), we have a `"prompt"` key which is a list of dictionaries. These dictionaries have a "role" and "parts" key. Here, we have a system prompt and a series of user/model interactions before finally having a user prompt. This acts as passing in a system prompt and conversation history.

Note that for each of these prompt dicts, we have `"model_name": "gemini-1.5-flash"`, besides `"id": 1` where we have `"model_name": "gemini-1.0-pro"`.

## Safety filters with Gemini API

With the Gemini API, it is possible to configure the safety filters (see the [safety settings docs](https://ai.google.dev/gemini-api/docs/safety-settings)). We can set the `"safety_filter"` key in the prompt dict where the options are:
- `"none"`: corresponds to "Block none" or `BLOCK_NONE`
- `"few"`: corresponds to "Block few" or `BLOCK_ONLY_HIGH`
- `"default"` or `"some"`: corresponds to "Block some" or `BLOCK_HIGH_AND_MEDIUM`
- `"most"`: corresponds to "Block most" or `BLOCK_LOW_AND_ABOVE`

In the example input file, we have set the `"safety_filter"` key to each of these options.

## Running the experiment

We now can run the experiment using the async method `process` which will process the prompts in the input file asynchronously. Note that a new folder named `timestamp-gemini-example` (where "timestamp" is replaced with the actual date and time of processing) will be created in the output directory and we will move the input file to the output directory. As the responses come in, they will be written to the output file and there are logs that will be printed to the console as well as being written to a log file in the output directory.

In [8]:
responses, avg_query_processing_time = await experiment.process()

Sending 6 queries  (attempt 1/3):   0%|          | 0/6 [00:00<?, ?query/s]

Waiting for responses  (attempt 1/3): 100%|██████████| 6/6 [00:01<00:00,  5.94query/s]


We can see that the responses are written to the output file, and we can also see them as the returned object. From running the experiment, we obtain prompt dicts where there is now a `"response"` key which contains the response(s) from the model.

For the case where the prompt is a list of strings, we see that the response is a list of strings where each string is the response to the corresponding prompt.

In [9]:
responses

[{'id': 0,
  'api': 'gemini',
  'model_name': 'gemini-1.5-flash',
  'prompt': 'How does technology impact us?',
  'safety_filter': 'none',
  'parameters': {'candidate_count': 1,
   'temperature': 1,
   'max_output_tokens': 100},
  'response': "Technology impacts us in countless ways, both positive and negative. Here's a breakdown of some key areas:\n\n**Positive Impacts:**\n\n* **Enhanced Communication:** Technology has revolutionized how we connect with others. Instant messaging, video calls, and social media platforms allow us to communicate with friends, family, and colleagues across the globe.\n* **Increased Efficiency and Productivity:**  Tools like computers, automation, and artificial intelligence (AI) streamline tasks, increase production, and improve overall efficiency in",
  'safety_attributes': {'HARM_CATEGORY_SEXUALLY_EXPLICIT': '1',
   'HARM_CATEGORY_HATE_SPEECH': '1',
   'HARM_CATEGORY_HARASSMENT': '1',
   'HARM_CATEGORY_DANGEROUS_CONTENT': '1',
   'blocked': '[False, Fal

Also notice how with the Gemini API, we record some additional information related to the safety attributes.

## Running the experiment via the command line

We can also run the experiment via the command line. The command is as follows (assuming that your working directory is the current directory of this notebook, i.e. `examples/gemini`):
```bash
prompto_run_experiment --file data/input/gemini-example.jsonl --max_queries 30
```